In [ ]:
import sys, os
sys.path.append(os.path.abspath('..'))
import sdf_helper as sh
import numpy as np
import matplotlib.pyplot as plt
from sdf_utils import *
import matplotlib as mpl

plt.rcParams.update({
    'font.family': ['Arial'],           # 字体优先使用 Arial，找不到时用 SimHei（支持中文）
    'font.size': 20,                              # 基础字体大小
    'axes.labelsize': 20,                         # 坐标轴标签字体大小
    'axes.titlesize': 20,                         # 坐标轴标题字体大小
    'xtick.labelsize': 18,                        # x 轴刻度标签字体大小
    'ytick.labelsize': 18,                        # y 轴刻度标签字体大小
    'legend.fontsize': 18,                        # 图例字体大小
    'mathtext.fontset': 'custom',                 # 使用自定义数学字体配置
    'mathtext.rm': 'Arial',                       # 数学文本正常字体用 Arial
    'mathtext.it': 'Arial:italic',                # 数学文本斜体用 Arial Italic
    'mathtext.bf': 'Arial:bold',                   # 数学文本粗体用 Arial Bold
    'xtick.direction': 'in',                      # x 轴刻度朝内
    'ytick.direction': 'in',                      # y 轴刻度朝内
})

# === 文件路径 ===
base_dir = '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch/epoch3d/Cpf-Work/cone/cone1-a=145-0.5微米'
file_name = 'field0009.sdf'
file_path = os.path.join(base_dir, file_name)

# === 读取电场数据 ===
result = load_field(file_path)
Ey = result['Ey']
Ez = result['Ez']
x = result['x']
y = result['y']
z = result['z']

# === 截取 x = 22.45 μm 平面 ===
x_index = np.argmin(np.abs(x - x_target))
Ey_slice = Ey[x_index, :, :]
Ez_slice = Ez[x_index, :, :]

# === 网格 & 极角 ===
Y, Z = np.meshgrid(y, z, indexing='ij')
phi = np.arctan2(Z, Y)

# === 笛卡尔 -> 柱坐标转换 ===
def cartesian_to_polar_field(Ey, Ez, phi):
    E_r = Ey * np.cos(phi) + Ez * np.sin(phi)
    E_phi = -Ey * np.sin(phi) + Ez * np.cos(phi)
    return E_r, E_phi

Er, Ephi = cartesian_to_polar_field(Ey_slice, Ez_slice, phi)

# === 重新读取 Ex 数据 ===
Ex = result['Ex']

# === 找到 y=2μm, z=0 对应索引 ===
y_index = np.argmin(np.abs(y - 2e-6))
z_index = np.argmin(np.abs(z - 0.0))

# === 提取 Ex(x), Ey(x), Ez(x) ===
Ex_line = Ex[:, y_index, z_index]
Ey_line = Ey[:, y_index, z_index]
Ez_line = Ez[:, y_index, z_index]

# === 计算对应的 phi（用于计算 Er）===
phi_line = np.arctan2(0.0, 2e-6)  # z = 0, y = 2e-6 → phi = 0

# === 计算 Er(x) ===
Er_line = Ey_line * np.cos(phi_line) + Ez_line * np.sin(phi_line)

# === 画图 ===
plt.figure(figsize=(7,5))
plt.plot(x * 1e6, Ex_line, label=r'$E_x$', color='b', linewidth=2)
plt.plot(x * 1e6, Er_line, label=r'$E_r$', color='r', linestyle='--', linewidth=2)
plt.xlabel('x (μm)')
plt.ylabel('Field (a.u.)')
plt.title(r'Fields at $y=2\ \mu m,\ z=0$')
plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig("/media/lan/Lexar/my_project/test/Ex_Er_line.pdf", format="pdf", dpi=600)
plt.xlim(10,30)
plt.show()
print(Er_line.max())
print(Ex_line.max())